importing libraries

In [19]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report




In [20]:
data = pd.read_csv('train_data.csv')

In [21]:
print(data.head)

<bound method NDFrame.head of       Loan_ID  Gender Married Dependents     Education Self_Employed  \
0    LP001002    Male      No          0      Graduate            No   
1    LP001003    Male     Yes          1      Graduate            No   
2    LP001005    Male     Yes          0      Graduate           Yes   
3    LP001006    Male     Yes          0  Not Graduate            No   
4    LP001008    Male      No          0      Graduate            No   
..        ...     ...     ...        ...           ...           ...   
609  LP002978  Female      No          0      Graduate            No   
610  LP002979    Male     Yes         3+      Graduate            No   
611  LP002983    Male     Yes          1      Graduate            No   
612  LP002984    Male     Yes          2      Graduate            No   
613  LP002990  Female      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0               5849            

In [22]:
X = data.drop('Loan_Status', axis=1)
y = data['Loan_Status']

In [23]:
le = LabelEncoder()
X_encoded = X.apply(lambda col: le.fit_transform(col.astype(str)), axis=0)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [42]:
# Drop rows with missing values
missing_indices_train = X_train.index[X_train.isnull().any(axis=1)]
X_train_no_missing = X_train.drop(missing_indices_train)
y_train_no_missing = y_train.drop(missing_indices_train)

missing_indices_test = X_test.index[X_test.isnull().any(axis=1)]
X_test_no_missing = X_test.drop(missing_indices_test)
y_test_no_missing = y_test.drop(missing_indices_test)

In [43]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [44]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_no_missing, y_train_no_missing)

DecisionTreeClassifier(random_state=42)

In [45]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_no_missing)
X_test_scaled = scaler.transform(X_test_no_missing)

# Model training
svm_model = SVC(random_state=42)
svm_model.fit(X_train_scaled, y_train_no_missing)

SVC(random_state=42)

In [46]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_scaled, y_train_no_missing)


KNeighborsClassifier()

In [47]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Model training
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_imputed, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [49]:
y_pred = model.predict(X_test)

print("Accuracy of Random Forest:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy of Random Forest: 0.7804878048780488

Classification Report:
               precision    recall  f1-score   support

           N       0.90      0.42      0.57        43
           Y       0.76      0.97      0.85        80

    accuracy                           0.78       123
   macro avg       0.83      0.70      0.71       123
weighted avg       0.81      0.78      0.75       123



In [50]:
y_pred_dt = dt_model.predict(X_test_no_missing)

# Print results
print("Decision Tree Model:")
print("Accuracy:", accuracy_score(y_test_no_missing, y_pred_dt))
print("\nClassification Report:\n", classification_report(y_test_no_missing, y_pred_dt))


Decision Tree Model:
Accuracy: 0.6260162601626016

Classification Report:
               precision    recall  f1-score   support

           N       0.46      0.40      0.42        43
           Y       0.70      0.75      0.72        80

    accuracy                           0.63       123
   macro avg       0.58      0.57      0.57       123
weighted avg       0.61      0.63      0.62       123



In [51]:
y_pred_svm = svm_model.predict(X_test_scaled)

# Print results
print("SVM Model:")
print("Accuracy:", accuracy_score(y_test_no_missing, y_pred_svm))
print("\nClassification Report:\n", classification_report(y_test_no_missing, y_pred_svm))

SVM Model:
Accuracy: 0.7886178861788617

Classification Report:
               precision    recall  f1-score   support

           N       0.95      0.42      0.58        43
           Y       0.76      0.99      0.86        80

    accuracy                           0.79       123
   macro avg       0.85      0.70      0.72       123
weighted avg       0.83      0.79      0.76       123



In [52]:
y_pred_knn = knn_model.predict(X_test_scaled)

# Print results
print("\nKNN Model:")
print("Accuracy:", accuracy_score(y_test_no_missing, y_pred_knn))
print("\nClassification Report:\n", classification_report(y_test_no_missing, y_pred_knn))



KNN Model:
Accuracy: 0.6504065040650406

Classification Report:
               precision    recall  f1-score   support

           N       0.50      0.12      0.19        43
           Y       0.66      0.94      0.78        80

    accuracy                           0.65       123
   macro avg       0.58      0.53      0.48       123
weighted avg       0.61      0.65      0.57       123



In [53]:

y_pred_lr = lr_model.predict(X_test_imputed)

# Print results
print("\nLogistic Regression Model with Imputed Data:")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))



Logistic Regression Model with Imputed Data:
Accuracy: 0.7886178861788617

Classification Report:
               precision    recall  f1-score   support

           N       0.95      0.42      0.58        43
           Y       0.76      0.99      0.86        80

    accuracy                           0.79       123
   macro avg       0.85      0.70      0.72       123
weighted avg       0.83      0.79      0.76       123

